[![View on GitHub](https://img.shields.io/badge/GitHub-View_on_GitHub-blue?logo=GitHub)](https://github.com/khuyentran1401/Data-science/blob/master/visualization/analyze_artificial_intelligence_industry.ipynb)

[<img src="https://deepnote.com/buttons/launch-in-deepnote.svg">](https://deepnote.com/project/Data-science-hxlyJpi-QrKFJziQgoMSmQ/%2FData-science%2Fvisualization%2Fanalyze_artificial_intelligence_industry.ipynb)

!wget -O 'artificial_intelligence.json' 'https://kg.diffbot.com/kg/dql_endpoint?type=query&token=your-token&query=type%3AOrganization+industries%3A%22Artificial+Intelligence+Companies%22&size=1000'

In [ ]:
import json

with open("/datasets/artificialintelligence/artificial_intelligence.json") as f:
    output = json.load(f)

In [ ]:
data = output["data"]

In [ ]:
data[0].keys()

dict_keys(['twitterUri', 'nbActiveEmployeeEdges', 'type', 'allNames', 'revenue', 'yearlyRevenues', 'logo', 'id', 'stock', 'nbOrigins', 'sicClassification', 'foundingDate', 'image', 'images', 'wikipediaUri', 'irsEmployerIdentificationNumbers', 'diffbotUri', 'nbIncomingEdges', 'nbEmployeesMin', 'ipo', 'parentCompany', 'angellistUri', 'name', 'motto', 'nbEmployeesMax', 'totalInvestment', 'allOriginHashes', 'linkedInUri', 'naicsClassification', 'nbEmployees', 'githubUri', 'isDissolved', 'importance', 'origin', 'description', 'homepageUri', 'founders', 'ceo', 'investments', 'blogUri', 'descriptors', 'isNonProfit', 'origins', 'isPublic', 'categories', 'crawlTimestamp', 'nbUniqueInvestors', 'facebookUri', 'secCentralIndexKeys', 'summary', 'types', 'boardMembers', 'allUris', 'nbLocations', 'crunchbaseUri', 'industries', 'allDescriptions', 'location', 'locations', 'subsidiaries'])

In [ ]:
from datetime import datetime

def get_founding_year(company: dict):
    date_str = company.get("foundingDate", {}).get("str")
    if date_str:
        return date_str.split("-")[0][1:]
    return date_str

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(
    {
        "company": [company.get("name") for company in data],
        "revenue": [company.get("revenue", {}).get("value") for company in data],
        "nbEmployees": [company.get("nbEmployees", None) for company in data],
        "founding_year": [get_founding_year(company) for company in data],
        "locations": [
            company.get("location", {}).get("country", {}).get("name")
            for company in data
        ],
        "isPublic": [company.get("isPublic") for company in data],
    }
)

df = df[df.notnull().all(axis=1)]
df["founding_year"] = df["founding_year"].astype(int)
df["year_from_now"] = df["founding_year"].apply(lambda row: datetime.now().year - row)

In [ ]:
df["isPublic"] = df["isPublic"].map({True: "public", False: "private"})

In [ ]:
df.head(5)

,company,revenue,nbEmployees,founding_year,locations,isPublic,year_from_now
0,Google,6.600100e+10,150000,1998,United States of America,public,23
1,ABB,3.431200e+10,60000,1988,Switzerland,public,33
2,Google,4.900000e+08,35000,1998,United States of America,private,23
3,Rockwell Automation Inc.,6.666000e+09,23500,1903,United States of America,public,118
4,Keyence,4.958000e+09,7500,1974,Japan,public,47


In [ ]:
df.to_csv("artificial_intelligence.csv", index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87197226-98be-42b2-8527-389082831299' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>